# ** ارائه یک چارچوب برای پاسخ به پرسش‌های قرآنی با ایجاد روابط پنج تایی **


In [ ]:
!pip install dadmatools
!pip install hazm


In [ ]:
!wget https://github.com/sobhe/hazm/releases/download/v0.5/resources-0.5.zip
!mkdir resources
!unzip /content/resources-0.5.zip -d /content/resources

*normalizer, tokenizer, lemmatizer, pos tagger, dependancy parser.*


In [ ]:
from dadmatools.models.normalizer import Normalizer

normalizer = Normalizer(
    full_cleaning=False,
    unify_chars=True,
    refine_punc_spacing=True,
    remove_extra_space=True,
    remove_puncs=False,
    remove_html=False,
    remove_stop_word=False,
    replace_email_with="<EMAIL>",
    replace_number_with=None,
    replace_url_with="",
    replace_mobile_number_with=None,
    replace_emoji_with=None,
    replace_home_number_with=None
)

text = """
اللّه، كه جز او معبودى نیست، زنده و برپادارنده است. نه خوابى سبك او را فرا گیرد و نه خوابى سنگین. آنچه در آسمان ها و آنچه در زمین است، از آنِ اوست. كیست آنكه جز به اذن او در پیشگاهش شفاعت كند. گذشته و آینده آنان را مى داند. و به چیزى از علم او احاطه پیدا نمى كنند مگر به مقدارى كه او بخواهد. كرسى (علم و قدرت) او آسمان ها و زمین را در برگرفته و نگهدارى آن دو بر او سنگین نیست و او والا و بزرگ است.
"""
#print('input text : ', text)
#print('out2 text when replace emails and remove urls : ', normalizer.normalize(text))

#full cleaning
#normalizer = Normalizer(full_cleaning=True)
##print('out2 text when using full_cleaning parameter', normalizer.normalize(text))

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from dadmatools.datasets import get_all_datasets_info, get_dataset_info
from dadmatools.datasets import ARMAN
from dadmatools.datasets import TEP
from dadmatools.datasets import PerSentLexicon
from dadmatools.datasets import FaSpell
from dadmatools.datasets import WikipediaCorpus
from dadmatools.datasets import PersianNer
from dadmatools.datasets import PersianNews
from dadmatools.datasets import PnSummary
from dadmatools.datasets import FarsTail
from dadmatools.datasets import SnappfoodSentiment
from dadmatools.datasets import get_all_datasets_info
from dadmatools.datasets import Peyma
from dadmatools.datasets import PerUDT
from dadmatools.datasets import PersianTweets
from pprint import pprint
import re, string 
import pandas as pd   
from collections import defaultdict
import spacy
from sklearn.manifold import TSNE
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
model = KeyedVectors.load_word2vec_format('drive/MyDrive/Thesis/model.bin', binary=True)
%matplotlib inline
from matplotlib import pyplot as plt
from hazm import *
import re
import numpy as np

In [ ]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [ ]:
!git clone https://github.com/facebookresearch/fastText.git
!cd fastText
!pip install fastText
from dadmatools.embeddings import get_embedding, get_all_embeddings_info, get_embedding_info
pprint(get_all_embeddings_info())
pprint(get_embedding_info('word2vec-conll'))
embedding = get_embedding('word2vec-conll')

In [ ]:
import dadmatools.pipeline.language as language

# here lemmatizer and pos tagger will be loaded
# as tokenizer is the default tool, it will be loaded even without calling
# pips = 'lem,pos' 
# pips = 'dep' 
pips = 'tok,lem,pos,dep,cons' 
nlp = language.Pipeline(pips)

# you can see the pipeline with this code
#print(nlp.analyze_pipes(pretty=True))

# doc is an SpaCy object
#doc = nlp(' این سوره یكصد و بیست آیه دارد و چند ماه پیش از رحلت پیامبر اكرم صلى الله علیه وآله نازل شده و هیچ یك از آیات آن نسخ نگشته است. این سوره به خاطر دعاى حضرت عیسى علیه السلام براى نزول مائده ى آسمانى كه در آیه ى 114 آمده، «مائده» نامیده شده است. البتّه به «عهد» و «عقود» نیز نام گذارى شده و در مجموع آیاتِ این سوره نسبت به یك پیمان شكنى مهم هشدار مى دهد.')

In [ ]:
def segmentation_verb(sen1):
  verb__=[]

  #print("segmentation_verb",sen1)
  Sentence=''
  l1=len(sen1[0])
  l2=l1
  #print(len(sen1[0]))
  for i in range(len(sen1[0])):
    if i< l1 and (sen1[0][i]['pos']=='VERB' and sen1[0][i]['pos']!='AUX') or ((sen1[0][i]['rel']=='cop' or sen1[0][i]['rel']=='ccomp') and sen1[0][i]['pos']=='AUX') or (sen1[0][i]['rel']=='aux' and sen1[0][i]['pos']=='AUX') or (sen1[0][i]['rel']=='root' and sen1[0][i]['pos']=='AUX') or (sen1[0][i]['text']=='،' and sen1[0][i+1]['rel']=='nsubj') :
      verb__.append(sen1[0][i]['id'])

 
  start=0
  sentn=[]
  for i in verb__:
    #print(i)
    for j in range(start, i):
      Sentence= Sentence + ' ' + sen1[0][j]['text']
    start=i
    sentn.append(Sentence)
    Sentence=''
  if start < (len(sen1[0])-2):
    for j in range(start, len(sen1[0])):
      Sentence= Sentence + ' ' + sen1[0][j]['text']
    sentn.append(Sentence)
    Sentence=''
    #print("sentnsentnsentnsentnsentnsentnsentnsentnsentn",sentn)

  if len(sentn)==1:
    sentn=[]

  return sentn



In [ ]:
def to_json1(pipelines, doc):
    dict_list = []
    for sent in doc._.sentences:
        sentence = []
        for i, d in enumerate(sent):
            dictionary = {}
            dictionary['id'] = i+1
            dictionary['text'] = d.text

            if 'dep' in pipelines: 
                dictionary['rel'] = d.dep_
                dictionary['root'] = d._.dep_arc
            sentence.append(dictionary)
        dict_list.append(sentence)
    return dict_list

In [ ]:
import dadmatools.pipeline.language as language

# here lemmatizer and pos tagger will be loaded
# as tokenizer is the default tool, it will be loaded even without calling
# pips = 'lem,pos' 
# pips = 'dep' 
pips = 'tok,lem,pos,dep' 
nlp = language.Pipeline(pips)

# you can see the pipeline with this code
#print(nlp.analyze_pipes(pretty=True))
text_="""چه کسی نوزاد خود را صبح با حالت نگران به رود انداخت"""
# doc is an SpaCy object
doc = nlp(text_)

Model fa_tokenizer exists in /root/.pernlp/fa_tokenizer.pt
Model fa_mwt exists in /root/.pernlp/fa_mwt.pt
Model fa_lemmatizer exists in /root/.pernlp/fa_lemmatizer.pt
Model parsbert exists in /root/.pernlp/parsbert.tar.gz
Model dependencyparser exists in /root/.pernlp/dependencyparser.pt
2023-02-19 22:46:46,744 loading file /usr/local/lib/python3.8/dist-packages/dadmatools/saved_models/dependencyparser/dependencyparser.pt
Model parsbert exists in /root/.pernlp/parsbert.tar.gz
Model postagger exists in /root/.pernlp/postagger.pt
2023-02-19 22:46:58,379 loading file /usr/local/lib/python3.8/dist-packages/dadmatools/saved_models/postagger/postagger.pt


[2023-02-19 22:47:06,379 INFO] [Ensembling dict with seq2seq lemmatizer...]


In [ ]:
output = language.to_json(pips, doc)
#print(output)

In [ ]:
def parser(text_):
 
  doc = nlp(text_)
  out2 = language.to_json(pips, doc)
  return out2



In [ ]:
def rootid_(out2):
  rootid=[]
  test_list = out2
  remov=[]
  l1=len(out2[0])
  l2=l1
  rootfirst=''
  for i in range(len(out2[0])):
    if i< l1 and (out2[0][i]['rel']=='root' or (out2[0][i]['pos']=='VERB' and out2[0][i]['root']==out2[0][i]['id'])):
      rootid.append(out2[0][i]['id'])
      if rootfirst=='':
        rootfirst=out2[0][i]['id']
  # اگر فعل و ریشه جمله تشخیص داده نشد به روش زیر عمل می کنیم
  if not rootid :
    rootlist=[]
    for i in range(len(out2[0])):
      rootlist.append(out2[0][i]['root'])
    idlist=most_frequent(rootlist)
    #print("list",rootlist,"most_frequent",idlist)
    rootid.append(out2[0][idlist-1]['id'])
    if rootfirst=='':
      rootfirst=out2[0][idlist-1]['id']
    out2[0][idlist-1]['root']=0
    #print(out2)
      

    
  #print("rootfirst",rootfirst)
  return rootid,rootfirst,out2

def most_frequent(List):
    return max(set(List), key = List.count)

  

In [ ]:
def nsubj_obl_obj(out2,rootfirst):
  rootnsubj=[]
  rootobl=[]
  rootobj=[]
  l1=len(out2[0])
  l2=l1
  List_obl = ['ADV','advmod','obl','obl:arg']
  
  #print("5%%%%3333333333%%%",rootnsubj)
  for i in range(len(out2[0])):
    #این شرط وجود اولین فاعل است if not rootnsubj
    if not rootnsubj and i< l1 and (out2[0][i]['rel']=='nsubj' or out2[0][i]['rel']=='nsubj:pass') :
      rootnsubj.append(out2[0][i]['id'])
   
    elif i< l1 and  (out2[0][i]['rel'] in List_obl) :
      rootobl.append(out2[0][i]['id'])
     
    elif i< l1  and out2[0][i]['rel']=='obj' :
      rootobj.append(out2[0][i]['id'])

  if not rootnsubj and (out2[0][0]['pos']=='NOUN' or out2[0][0]['pos']=='PROPN') and out2[0][0]['rel']=='nmod' and out2[0][0]['root']==rootfirst :
    rootnsubj.append(out2[0][0]['id'])
    out2[0][0]['rel']='nsubj'
    #print("5%%%%%%%",rootnsubj)
  #print("5%%%%%%%1111111111",rootnsubj)
  rootobl1=[]
  rootobl2=[]
  rootobl3=[]
  rootobl4=[]
  if len(rootobl)>0:
    rootobl1.append(rootobl[0])
  if len(rootobl)>1:
    rootobl2.append(rootobl[1])
  if len(rootobl)>2:
    rootobl3.append(rootobl[2])
  if len(rootobl)>3:
    rootobl4.append(rootobl[3])  
  return out2,rootnsubj,rootobj,rootobl1,rootobl2,rootobl3,rootobl4


In [ ]:
def duplicates(list1):
  seen = set()
  uniq = []
  for x in list1:
      if x not in seen:
          uniq.append(x)
          seen.add(x)
  return uniq

In [ ]:
def nsubj_obl_obj_full(out2,rootnsubj,rootobl1,rootobl2,rootobl3,rootobl4,rootobj,rootid):
  for r in range(len(out2[0])):
    l1=len(out2[0])
    l2=l1
    remov=[]
    for i in range(len(out2[0])):
      for j in range(len(rootnsubj)):
        if i< l1 and out2[0][i]['root']==rootnsubj[j] and out2[0][i]['id'] != rootnsubj[j]:
          if out2[0][i]['id'] > rootnsubj[j] :
            rootnsubj.append(out2[0][i]['id'])
            rootnsubj=duplicates(rootnsubj)
          
          elif out2[0][i]['id'] < rootnsubj[j] :
            rootnsubj.append(out2[0][i]['id'])
            rootnsubj=duplicates(rootnsubj)
          

      for j in range(len( rootobl1)):
        if i< l1 and out2[0][i]['root']== rootobl1[j] and out2[0][i]['id'] != rootobl1[j]:
          if out2[0][i]['id'] >  rootobl1[j] :
             rootobl1.append(out2[0][i]['id'])
             rootobl1=duplicates( rootobl1)
          
          elif out2[0][i]['id'] <  rootobl1[j] :
             rootobl1.append(out2[0][i]['id'])
             rootobl1=duplicates( rootobl1)
        
      for j in range(len( rootobl2)):
        if i< l1 and out2[0][i]['root']== rootobl2[j] and out2[0][i]['id'] != rootobl2[j]:
          if out2[0][i]['id'] >  rootobl2[j] :
             rootobl2.append(out2[0][i]['id'])
             rootobl2=duplicates( rootobl2)
          
          elif out2[0][i]['id'] <  rootobl2[j] :
             rootobl2.append(out2[0][i]['id'])
             rootobl2=duplicates( rootobl2)

      for j in range(len( rootobl3)):
        if i< l1 and out2[0][i]['root']== rootobl3[j] and out2[0][i]['id'] != rootobl3[j]:
          if out2[0][i]['id'] >  rootobl3[j] :
             rootobl3.append(out2[0][i]['id'])
             rootobl3=duplicates( rootobl3)
          
          elif out2[0][i]['id'] <  rootobl3[j] :
             rootobl3.append(out2[0][i]['id'])
             rootobl3=duplicates( rootobl3)

      for j in range(len( rootobl4)):
        if i< l1 and out2[0][i]['root']== rootobl4[j] and out2[0][i]['id'] != rootobl4[j]:
          if out2[0][i]['id'] >  rootobl4[j] :
             rootobl4.append(out2[0][i]['id'])
             rootobl4=duplicates( rootobl4)
          
          elif out2[0][i]['id'] <  rootobl4[j] :
             rootobl4.append(out2[0][i]['id'])
             rootobl4=duplicates( rootobl4)         

      for j in range(len(rootobj)):
        if i< l1 and out2[0][i]['root']==rootobj[j] and out2[0][i]['id'] != rootobj[j]:
          
          if out2[0][i]['id'] > rootobj[j] :
            rootobj.append(out2[0][i]['id'])
            rootobj=duplicates(rootobj)
        
          elif out2[0][i]['id'] < rootobj[j] :
            rootobj.append(out2[0][i]['id'])
            rootobj=duplicates(rootobj)
        


      for j in range(len(rootid)):
        if i< l1 and out2[0][i]['root']==rootid[j] and out2[0][i]['id'] != rootid[j] and (out2[0][i]['rel'] not in ('nsubj','obj','obl:arg','obl')):
          if out2[0][i]['id'] > rootid[j] :
            rootid.append(out2[0][i]['id'])
            rootid=duplicates(rootid)
          
          elif out2[0][i]['id'] < rootid[j] :
            rootid.append(out2[0][i]['id'])
            rootid=duplicates(rootid)
  return rootnsubj,rootobj,rootobl1,rootobl2,rootobl3,rootobl4,rootid

In [ ]:
def pars_text(out2,rootnsubj,rootobl1,rootobl2,rootobl3,rootobl4,rootobj,rootid): 
  nsubj=''
  obl1=''
  obl2=''
  obl3=''
  obl4=''
  obj=''
  verb=''
  
  for i in range(len(out2[0])):
    for j in rootid:
      if  out2[0][i]['id']==j :
        verb= verb + ' ' + out2[0][i]['text']
    for j in  rootobl1:
      if  out2[0][i]['id']==j :
        obl1= obl1 + ' ' + out2[0][i]['text']
    for j in  rootobl2:
      if  out2[0][i]['id']==j :
        obl2= obl2 + ' ' + out2[0][i]['text']
    for j in  rootobl3:
      if  out2[0][i]['id']==j :
        obl3= obl3 + ' ' + out2[0][i]['text']
    for j in  rootobl4:
      if  out2[0][i]['id']==j :
        obl4= obl4 + ' ' + out2[0][i]['text']
    for j in rootnsubj:
      if  out2[0][i]['id']==j :
        nsubj= nsubj + ' ' + out2[0][i]['text']
    for j in rootobj:
      if  out2[0][i]['id']==j :
        obj= obj + ' ' + out2[0][i]['text']

  return nsubj,obj,obl1,obl2,obl3,obl4,verb


In [ ]:
def wordcloud_def(text,stop_words):
  word_tokens = word_tokenize(text)
  
  text='' 
  for w in word_tokens:
      if w not in stop_words:
          text=text + ' ' + w
  return(text)

In [ ]:
import codecs
def stopwords():
  nmz = Normalizer()
  stop_words = "\n".join(
      sorted(
          list(
              set(
                  [
                      nmz.normalize(w) for w in codecs.open('/content/drive/MyDrive/Thesis/persian', encoding='utf-8').read().split('\n') if w
                  ]
              )
          )
      )
  )
  return(stop_words)

In [ ]:
text="در دانشگاه"
stop_words=stopwords()
wordcloud_def(text,stop_words)

' دانشگاه'

In [ ]:
def adverb_type(text):
  from dadmatools.models.normalizer import Normalizer
  stop_words=stopwords()
  text=wordcloud_def(text,stop_words)
  normalized_text = normalizer.normalize(text)
  #print("normalized_text",normalized_text)
  typeAdverbs=''
  if normalized_text !='':
    typeAdverbs=''
    doc=nlp( normalized_text)
    #print(doc)
    sentences = doc._.sentences
    #print("sentences::::",sentences)
    for sentence in sentences:
      out3=parser(str(sentence))
      #print(out3)
      #print("text111111111111111:::",out3)
      noun=[]
      for i in range(len(out3[0])):
        if out3[0][i]['pos']=='NOUN' :
          noun.append(out3[0][i]['text'])
#اگر در جمله اسم وجود نداشت 
      if not noun:
        for i in range(len(out3[0])):
          noun.append(out3[0][i]['text'])
    #  for sentence in moun:
  #       text = sentence
      #print(noun)
      for token in noun:
          #print(str(token))
          typeAdverbs=Adverb(str(token))
          if typeAdverbs=='T' or typeAdverbs=='P' or typeAdverbs=='M':
            #print("typeAdverbs",typeAdverbs)
            break
  
  return typeAdverbs



In [ ]:
# CREATING THE TABLE
import sqlite3

conn = sqlite3.connect('/content/drive/MyDrive/Thesis/QandA.db')
#print("Opened database successfully");

#conn.execute('''DROP TABLE IF EXISTS knowledge_data1;''')
conn.commit()
conn.execute('''
CREATE TABLE IF NOT EXISTS knowledge_data3(id INTEGER PRIMARY KEY AUTOINCREMENT, sentence text, nsubj text, 
                      obj text, 
                      oblTime text, 
                      oblPlace text, 
                      oblManner text, 
                      oblNull text, 
                      verb text);''')


conn.commit()

#print("Table created successfully");

conn.close()

In [ ]:
def quintuplet(doc,id,flag):
 # doc = 'سوره بقره یكصد و بیست آیه دارد و چند ماه پیش از رحلت پیامبر اكرم نازل شده و هیچ یك از آیات آن نسخ نگشته است .'
  #print(doc)
  out1=parser(doc)
  #print(out1)
  id1=id
  rootid,rootfirst,out2=rootid_(out1)
  out2,rootnsubj,rootobj,rootobl1,rootobl2,rootobl3,rootobl4=nsubj_obl_obj(out2,rootfirst)
  out1=parser(doc)

  nsubj,obj,obl1,obl2,obl3,obl4,verb=pars_text(out2,rootnsubj,rootobl1,rootobl2,rootobl3,rootobl4,rootobj,rootid)
  #print('فاعل جمله : ','*****',nsubj,'*****')
  #print('مفعول جمله : ','*****',obj,'*****')
  #print('قید اول جمله : ','*****',obl1,'*****')
  #print('قید دوم جمله : ','*****',obl2,'*****')
  #print('قید سوم جمله : ','*****',obl3,'*****')
  #print('قید چهارم جمله : ','*****',obl4,'*****')
  #print('فعل جمله : ','*****',verb,'*****')

  rootnsubj,rootobj,rootobl1,rootobl2,rootobl3,rootobl4,rootid=nsubj_obl_obj_full(out2,rootnsubj,rootobl1,rootobl2,rootobl3,rootobl4,rootobj,rootid)
  rootnsubj.sort()
  rootobl1.sort()
  rootobl2.sort()
  rootobl3.sort()
  rootobl4.sort()
  rootobj.sort()
  rootid.sort()
  #print(rootnsubj)
  #print(rootobj)
  #print(rootobl1)
  #print(rootobl2)
  #print(rootobl3)
  #print(rootobl4)
  #print(rootid)

  out1=parser(doc)
  nsubj,obj,obl1,obl2,obl3,obl4,verb=pars_text(out2,rootnsubj,rootobl1,rootobl2,rootobl3,rootobl4,rootobj,rootid)
  #print('فاعل جمله : ','*****',nsubj,'*****')
  #print('مفعول جمله : ','*****',obj,'*****')
  #print('قید اول جمله : ','*****',obl1,'*****')
  #print('قید دوم جمله : ','*****',obl2,'*****')
  #print('قید سوم جمله : ','*****',obl3,'*****')
  #print('قید چهارم جمله : ','*****',obl4,'*****')
  #print('فعل جمله : ','*****',verb,'*****')
  oblTime =''
  oblPlace =''
  oblManner =''
  oblNull =''
  oblNull1=''
  typeAdv =''
  #Prediction the type of adverbs
  
  if obl1 !='':
    
    typeAdv=adverb_type(obl1)
    
    if typeAdv=='T' and oblTime=='':
      oblTime=obl1
    elif typeAdv=='P' and oblPlace=='':
      oblPlace=obl1
    elif typeAdv=='M' and oblManner=='':
      oblManner=obl1
    elif typeAdv=='N' :
      oblNull=obl1
      oblNull1=oblNull1 + obl1
  
#############
  if obl2 !='':
    #print("obl2",obl2)
    typeAdv=adverb_type(obl2)
    #print("typeAdv,",typeAdv)
    if typeAdv=='T' and oblTime=='':
      oblTime=obl2
    elif typeAdv=='P' and oblPlace=='':
      oblPlace=obl2
    elif typeAdv=='M' and oblManner=='':
      oblManner=obl2
    elif typeAdv=='N' :
      oblNull=obl2
      oblNull1=oblNull1 + obl2
  

#############
  if obl3 !='':
    typeAdv=adverb_type(obl3)
    if typeAdv=='T' and oblTime=='':
      oblTime=obl3
    elif typeAdv=='P' and oblPlace=='':
      oblPlace=obl3
    elif typeAdv=='M' and oblManner=='':
      oblManner=obl3
    elif typeAdv=='N' :
      oblNull=obl3
      oblNull1=oblNull1 +  obl3
      
#############
  if obl4 !='':
    typeAdv=adverb_type(obl4)
    if typeAdv=='T' and oblTime=='':
      oblTime=obl4
    elif typeAdv=='P' and oblPlace=='':
      oblPlace=obl4
    elif typeAdv=='M' and oblManner=='':
      oblManner=obl4
    elif typeAdv=='N' :
      oblNull=obl4
      oblNull1=oblNull1  + obl4
  # INSERTING VALUES
  #print("oblPlace",oblPlace)
  if flag=='1':
    conn = sqlite3.connect('/content/drive/MyDrive/Thesis/QandA.db')
    cursor = conn.cursor()
    sqlite_insert_with_param = """INSERT INTO knowledge_data1
                      ( sentence, nsubj, obj, oblTime, oblPlace, oblManner, oblNull, verb) 
                      VALUES ( ?, ?, ?, ?, ?, ?, ?, ?);"""

    data_tuple = (doc, nsubj, obj, oblTime, oblPlace, oblManner, oblNull1, verb)
    cursor.execute(sqlite_insert_with_param, data_tuple)
    conn.commit()

    #print("@@@@@@@@@@@@@@@@@@@@@@@@@@222222",oblNull1)
    #print(out1)
  else:
    return id, doc, nsubj, obj, oblTime, oblPlace, oblManner, oblNull1, verb


In [ ]:
def Adverb(word):
  try:
      string=word
      str=[]
      
      str.append(string)
      #sim1= embedding.top_nearest([string],20)
      sim1=model.wv.most_similar(string,topn=20)
      
      for index, row in enumerate(sim1):
        ##print(sim1[index][0])
        str.append(sim1[index][0])
    ####################################1
      
      list_Time=[]
      for (num,item) in enumerate(str):
          ##print(num,item)
          string1 = item   
          # opening a text file
          file_Time = open("/content/drive/MyDrive/Thesis/Adverb_of_Time.txt", "r")
          # setting flag and index to 0
          flag_Time = 0
          index_Time = 0      
          # Loop through the file line by line
          for line_Time in file_Time:  
              index_Time = index_Time +1           
              # checking string is present in line or not
              if string1 in line_Time:     
                flag_Time = 1
                break 
                    
          # checking condition for string found or not
          if flag_Time == 0: 
            flag_Time=0
          # #print('String', string1 , 'Not Found') 
          else: 
            ##print('String', string1, 'Found In Line', index_Time)
            list_Time.append(num+1)
          # closing text file    
          file_Time.close() 

    # for item in list_Time:
          ##print(item)
    ##################################2
      list_place=[]
      for (num,item) in enumerate(str):
          ##print(num,item)
          string1 = item
          file_place = open("/content/drive/MyDrive/Thesis/Adverb_of_place.txt", "r")   
          flag_place = 0
          index_place = 0 
          # Loop through the file line by line
          for line_place in file_place:  
              index_place = index_place +1
              # checking string is present in line or not
              if string1 in line_place:
                flag_place = 1
                break   
          # checking condition for string found or not
          if flag_place == 0: 
            flag_place=0
            ##print('String', string1 , 'Not Found') 
          else: 
            ##print('String', string1, 'Found In Line', index_place)
            list_place.append(num+1)
            
          # closing text file    
          file_place.close() 
      #for item in list_place:

          ##print(item)
    ################################3
      list_manner=[]
      for (num,item) in enumerate(str):
      #   #print(num,item)
          string1 = item
          file_manner = open("/content/drive/MyDrive/Thesis/Adverb_of_manner.txt", "r")   
          flag_manner = 0
          index_manner = 0
          # Loop through the file line by line
          for line_manner in file_manner:  
              index_manner = index_manner +1
              # checking string is present in line or not
              if string1 in line_manner:
                flag_manner = 1
                break   
          # checking condition for string found or not
          if flag_manner == 0: 
            flag_manner=0
            ##print('String', string1 , 'Not Found') 
          else: 
            ##print('String', string1, 'Found In Line', index_manner)
            list_manner.append(num+1)
          # closing text file    
          file_manner.close() 
    #  for item in list_manner:
      #   #print(item)
    #################################prediction_type_adverb
      typeAdverb=''
      if len(list_Time) > len(list_place) and len(list_Time) > len(list_manner):
        #print('قید زمان است')
        typeAdverb='T'
      elif len(list_Time) < len(list_place) and len(list_manner) < len(list_place):
        #print('قید مکان است')
        typeAdverb='P'
      elif len(list_Time) < len(list_manner) and len(list_place) < len(list_manner):
        #print('قید حالت است')
        typeAdverb='M'
      elif len(list_Time)==0 and len(list_place)==0 and len(list_manner)==0:
        #print('قید مکان یا زمان یا حالت نیست')
        typeAdverb='N'
      elif len(list_Time)==0 and len(list_place)==len(list_manner):
        for x in range(0, len(list_place)):
          if  list_manner[x] > list_place[x]:
            #print('قید مکان است')
            typeAdverb='P'
            break
          if list_manner[x] < list_place[x]:
            #print('قید حالت است')
            typeAdverb='T'
            break

      elif len(list_place)==0 and len(list_Time)==len(list_manner):
        for x in range(0, len(list_Time)):
          if  list_manner[x] > list_Time[x]:
            #print('قید زمان است')
            typeAdverb='P'
            break
          if list_manner[x] < list_Time[x]:
            #print('قید حالت است')
            typeAdverb='T'
            break

      elif len(list_manner)==0 and len(list_Time)==len(list_place):
        for x in range(0, len(list_Time)):
          if  list_place[x] > list_Time[x]:
            #print('قید زمان است')
            typeAdverb='P'
            break
          if list_place[x] < list_Time[x]:
            #print('قید مکان است')
            typeAdverb='T'
            break
  except KeyError:
      return 'N'
  else:  
      return typeAdverb

In [ ]:
def dadma_first(text,flag):

  id=0

  text1=[]

  text1=sent_tokenize(text)
  #for sent in text1:
  #print("text1,text1",text)
  senn=[]
  
  for sent in text1:
    #print('جمله:::::::',sent)

    id=id + 1
    quintuplet(sent,id,flag)

    doc1 = nlp(sent)
    out1 = language.to_json(pips, doc1)
    senn=segmentation_verb(out1)
    #print("sennnnn",senn)
    #print("%%%%%%%%%%%%%%%%%5")
    #print("%%%%%%%%%%%%%%%%%5")
    #print("%%%%%%%%%%%%%%%%%5")
    if senn:
      for sen in senn:
        #print(sen,'verbbbbbbbbbb')
        id=id + 1
        quintuplet(sen,id,flag)
      #  doc1 = nlp(str(sen))
      #  out21 = language.to_json(pips, doc1)
      #  #print(out21)

    # doc1 = nlp(str(sent))
    # out21 = language.to_json(pips, doc1)
    # #print(out21)



  #print(id)

**هضم**

In [ ]:
def segmentation_verb_hazm(txt):

  tagger = POSTagger(model='resources/postagger.model')
  lemmatizer = Lemmatizer()
  parser = DependencyParser(tagger=tagger, lemmatizer=lemmatizer)

  #نرمالسازی و تمیز کردن جملات

  #print(txt)
  #تبدیل متن به جملات
  text1=sent_tokenize(txt)
  #print(text1)
  #print('##########################')
  list1=[]

  sentences = parser.parse(word_tokenize(txt))
  Sentence=''
  sentn=[]
  verb__=[]
  list_hazm=[]
  list_word=word_tokenize(txt)
  for a in parser.parse(word_tokenize(txt)).triples():
    aa=a
    list_hazm.append(a)
    #print(a)
  #print('VERB',list_hazm)
  
  for i in range(len(list_hazm)):
    if list_hazm[i][0][1]=='V' or (len(list_hazm)>i+1 and list_hazm[i+1][1]=='SBJ'):
      verb__.append(list_hazm[i][0][0])

  for i in list_word:
    if i not in verb__:
      Sentence= Sentence + ' ' + i
    else:
      Sentence= Sentence + ' ' + i
      sentn.append(Sentence)
      Sentence=''

  if len(sentn)==1:
    sentn=[]

  return sentn



In [ ]:
def hazm_first(text,flag):
  id=0

  text1=[]
  from hazm import sent_tokenize, word_tokenize

  text1=sent_tokenize(text)
  #for sent in text1:
  #print("text1,text1",text)
  senn=[]
  flag='1'
  for sent in text1:
    #print('جمله:::::::',sent)

    id=id + 1
    quintuplethazm(sent,id,flag)

    senn=segmentation_verb_hazm(sent)
    #print("sennnnn",senn)
    #print("%%%%%%%%%%%%%%%%%5")
    #print("%%%%%%%%%%%%%%%%%5")
    #print("%%%%%%%%%%%%%%%%%5")
    if senn:
      for sen in senn:
        #print(sen,'verbbbbbbbbbb')
        id=id + 1
        quintuplethazm(sen,id,flag)
      #  doc1 = nlp(str(sen))
      #  out21 = language.to_json(pips, doc1)
      #  #print(out21)

    # doc1 = nlp(str(sent))
    # out21 = language.to_json(pips, doc1)
    # #print(out21)



  #print(id)


In [ ]:
def hazmparse(txt):  
  try:  
      tagger = POSTagger(model='resources/postagger.model')
      lemmatizer = Lemmatizer()
      parser = DependencyParser(tagger=tagger, lemmatizer=lemmatizer)

      #نرمالسازی و تمیز کردن جملات

      #print(txt)
      #تبدیل متن به جملات
      text1=sent_tokenize(txt)
      #print(text1)
      #print('##########################')
      list1=[]

      sentences = parser.parse(word_tokenize(txt))
      #print(sentences.tree())

      #print('##########################')
      SBJ=''
      OBJ=''
      VERB=''

      list_hazm=[]
      for a in parser.parse(word_tokenize(txt)).triples():
        aa=a
        list_hazm.append(a)
        #print(a)
      #print('VERB',list_hazm)
      
      

      ADV1=''
      ADV2=''
      ADV3=''
      list_adv1=[]
      list_adv2=[]
      list_adv3=[]
      list_sbj=[]
      list_obj=[]
      ADV1_clear=1
      ADV2_clear=1
      ADV3_clear=1
      List_ADV = ['ADV','VPP']
      for i in range(len(list_hazm)):
        if list_hazm[i][0][1]=='V':
          VERB=list_hazm[i][0][0]


      for i in range(len(list_hazm)):
    #    #print(list_hazm[i][1])
    #    if list[i][1]=='SBJ':
    #      SBJ= list[i][2][0]
    #      if len(list)>i+1 and list[i+1][2][1]not in List_ADV and list[i+1][0][0]==list[i][2][0]:
    #        #print(list[i+1][2][0])
    #        SBJ= SBJ + ' '+ list[i+1][2][0]
    #        if len(list)>i+2 and list[i+2][0][0]==list[i+1][2][0]:
    #          SBJ= SBJ + ' '+ list[i+2][2][0]
    #      if len(list)>i+1 and list[i+1][2][1]in List_ADV:
    #        ADV1= list[i+1][2][0]
    #        if len(list)>i+2 and list[i+2][0][0]==list[i+1][2][0]:
    #          ADV1= ADV1 + ' '+ list[i+2][2][0]
    #          if len(list)>i+3 and list[i+3][0][0]==list[i+2][2][0]:
    #            ADV1= ADV1 + ' '+ list[i+3][2][0]
    #      continue
        if list_hazm[i][1]=='SBJ' or list_hazm[i][0][0] in list_sbj:
          list_sbj.append(list_hazm[i][2][0])
          SBJ= SBJ + ' '+ list_hazm[i][2][0]
          if len(list_hazm)==i+1 or list_hazm[i+1][0][1]=='V':
            list_sbj.clear()
          continue

        elif list_hazm[i][1]=='OBJ' or list_hazm[i][0][0] in list_obj:
          list_obj.append(list_hazm[i][2][0])
          OBJ= OBJ + ' '+ list_hazm[i][2][0]
          if len(list_hazm)==i+1 or list_hazm[i+1][0][1]=='V':
            list_obj.clear()
          continue
          
        elif  ADV1_clear==1  and (list_hazm[i][1] in List_ADV or list_hazm[i][0][0] in list_adv1):
          list_adv1.append(list_hazm[i][2][0])
          ADV1= ADV1 + ' '+ list_hazm[i][2][0] 
          if len(list_hazm)==i+1 or list_hazm[i+1][0][1]=='V':
            list_adv1.clear()
            ADV1_clear=0                                       
          continue

        elif  ADV2_clear==1  and (list_hazm[i][1] in List_ADV or list_hazm[i][0][0] in list_adv2):
          list_adv2.append(list_hazm[i][2][0])
          ADV2= ADV2 + ' '+ list_hazm[i][2][0] 
          if len(list_hazm)==i+1 or list_hazm[i+1][0][1]=='V':
            list_adv2.clear()  
            ADV2_clear=0                                      
          continue

        elif  ADV3_clear==1 and (list_hazm[i][1] in List_ADV or list_hazm[i][0][0] in list_adv3):
          list_adv3.append(list_hazm[i][2][0])
          ADV3= ADV3 + ' '+ list_hazm[i][2][0] 
          if len(list_hazm)==i+1 or list_hazm[i+1][0][1]=='V':
            list_adv3.clear()  
            ADV3_clear=0                                      
          continue

        else:
          if list_hazm[i][0][0]==VERB and list_hazm[i][2][1]!='CONJ':
            #print(list_hazm[i][2][0])
            VERB=list_hazm[i][2][0] + ' ' + VERB


      ADV4=''
      #print('SBJ: ',SBJ)
      #print('OBJ: ',OBJ)
      #print('ADV1: ',ADV1)
      #print('ADV2: ',ADV2)
      #print('ADV3: ',ADV3)
      #print('VERB: ',VERB)
      

  except IndexError:
      return '','','','','','',''
  else:  
      return SBJ,OBJ,ADV1,ADV2,ADV3,ADV4,VERB

In [ ]:
def quintuplethazm(doc,id,flag):
 # doc = 'سوره بقره یكصد و بیست آیه دارد و چند ماه پیش از رحلت پیامبر اكرم نازل شده و هیچ یك از آیات آن نسخ نگشته است .'
  #print(doc)
 
  id1=id

  nsubj,obj,obl1,obl2,obl3,obl4,verb=hazmparse(doc)
  #print('فاعل جمله : ','*****',nsubj,'*****')
  #print('مفعول جمله : ','*****',obj,'*****')
  #print('قید اول جمله : ','*****',obl1,'*****')
  #print('قید دوم جمله : ','*****',obl2,'*****')
  #print('قید سوم جمله : ','*****',obl3,'*****')
  #print('قید چهارم جمله : ','*****',obl4,'*****')
  #print('فعل جمله : ','*****',verb,'*****')
  #print('hazm*********************************************hazm')
  oblTime =''
  oblPlace =''
  oblManner =''
  oblNull =''
  oblNull1=''
  typeAdv =''
  #Prediction the type of adverbs
  
  if obl1 !='':
    
    typeAdv=adverb_type(obl1)
    
    if typeAdv=='T' and oblTime=='':
      oblTime=obl1
    elif typeAdv=='P' and oblPlace=='':
      oblPlace=obl1
    elif typeAdv=='M' and oblManner=='':
      oblManner=obl1
    elif typeAdv=='N' :
      oblNull=obl1
      oblNull1=oblNull1 + obl1
  
#############
  if obl2 !='':
    #print("obl2",obl2)
    typeAdv=adverb_type(obl2)
    #print("typeAdv,",typeAdv)
    if typeAdv=='T' and oblTime=='':
      oblTime=obl2
    elif typeAdv=='P' and oblPlace=='':
      oblPlace=obl2
    elif typeAdv=='M' and oblManner=='':
      oblManner=obl2
    elif typeAdv=='N' :
      oblNull=obl2
      oblNull1=oblNull1 + obl2
  

#############
  if obl3 !='':
    typeAdv=adverb_type(obl3)
    if typeAdv=='T' and oblTime=='':
      oblTime=obl3
    elif typeAdv=='P' and oblPlace=='':
      oblPlace=obl3
    elif typeAdv=='M' and oblManner=='':
      oblManner=obl3
    elif typeAdv=='N' :
      oblNull=obl3
      oblNull1=oblNull1 +  obl3
      
#############
  if obl4 !='':
    typeAdv=adverb_type(obl4)
    if typeAdv=='T' and oblTime=='':
      oblTime=obl4
    elif typeAdv=='P' and oblPlace=='':
      oblPlace=obl4
    elif typeAdv=='M' and oblManner=='':
      oblManner=obl4
    elif typeAdv=='N' :
      oblNull=obl4
      oblNull1=oblNull1  + obl4
  # INSERTING VALUES
  #print("oblPlace",oblPlace)
  if flag=='1':
    conn = sqlite3.connect('/content/drive/MyDrive/Thesis/QandA.db')
    cursor = conn.cursor()
    sqlite_insert_with_param = """INSERT INTO knowledge_data1
                      (sentence, nsubj, obj, oblTime, oblPlace, oblManner, oblNull, verb) 
                      VALUES (?, ?, ?, ?, ?, ?, ?, ?);"""

    data_tuple = (doc, nsubj, obj, oblTime, oblPlace, oblManner, oblNull1, verb)
    cursor.execute(sqlite_insert_with_param, data_tuple)
    conn.commit()

    #print("@@@@@@@@@@@@@@@@@@@@@@@@@@222222",oblNull1)
    #print(out1)
  else:
    return id, doc, nsubj, obj, oblTime, oblPlace, oblManner, oblNull1, verb


In [ ]:
from hazm import sent_tokenize, word_tokenize
text="""همۀ مردان و زنان باایمان و هر مؤمنی را هم که وارد خانه‌ام می‌شود بیامرز"""
text= text.replace("\u200c", " " )
normalizer = Normalizer()
text = normalizer.normalize(text)
flag='1'
#dadma_first(text,flag)
#print('dadma_first_dadma_first****************************')
hazm_first(text,flag)
#print('hazm_first_hazm_first****************************')


In [ ]:
conn = sqlite3.connect('/content/drive/MyDrive/Thesis/QandA.db')

cursor = conn.execute(''' SELECT id, sentence, nsubj, obj, oblTime, oblPlace, oblManner, oblNull, verb
                          FROM knowledge_data_full;''')

for row in cursor:
  #print(row[2])
  print(row)
conn.close()

In [ ]:
!pip install sentence-transformers
from sentence_transformers import SentenceTransformer

# Load the BERT model. Various models trained on Natural Language Inference (NLI) https://github.com/UKPLab/sentence-transformers/blob/master/docs/pretrained-models/nli-models.md and 
# Semantic Textual Similarity are available https://github.com/UKPLab/sentence-transformers/blob/master/docs/pretrained-models/sts-models.md
#model = SentenceTransformer('HooshvareLab/bert-fa-zwnj-base')
#model = SentenceTransformer('HooshvareLab/bert-fa-base-uncased-clf-digimag')
model1 = SentenceTransformer('HooshvareLab/bert-fa-base-uncased-clf-persiannews')
#model1 = SentenceTransformer('HooshvareLab/bert-fa-zwnj-base')

In [ ]:
#from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
#from sentence-transformers import SentenceTransformer
import scipy
import csv

#model = SentenceTransformer('HooshvareLab/bert-fa-zwnj-base')
#model = SentenceTransformer('HooshvareLab/bert-fa-base-uncased-clf-digimag')
#model1 = SentenceTransformer('HooshvareLab/bert-fa-base-uncased-clf-persiannews')



In [ ]:
conn = sqlite3.connect('/content/drive/MyDrive/Thesis/QandA1.db')
cursor = conn.cursor()
sql_delete_query = """INSERT INTO knowledge_data_full
                      (sentence, nsubj, obj, oblTime, oblPlace, oblManner, oblNull, verb) SELECT sentence, nsubj, obj, oblTime, oblPlace, oblManner, oblNull, verb from knowledge_data1;"""

cursor.execute(sql_delete_query)
#print("Record deleted successfully ")
conn.commit()

In [ ]:
conn = sqlite3.connect('/content/drive/MyDrive/Thesis/QandA.db')

cursor = conn.execute(''' SELECT id, sentence
                          FROM knowledge_data_full;''')
list_sent=[]
for row in cursor:
  #print(row[1])
  print(row[0])
  list_sent.append(row[1])
  
sentence_embeddings = model1.encode(list_sent)
#sen_embedding2.shape
conn.close()

In [ ]:

def embedding_Comparison(item1,item2):
  sen_embedding3 = model1.encode(item1)
  sen_embedding4 = model1.encode(item2)
  sen_embedding3.shape
  sen_embedding4.shape

  from sklearn.metrics.pairwise import cosine_similarity
  #let's calculate cosine similarity for sentence 0:
  Comparison=cosine_similarity(
      [sen_embedding3],
      [sen_embedding4]
  )
  return(Comparison[0][0])



In [ ]:
def sort1(tex,sentence):
  tex=word_tokenize (tex)
  sentence=word_tokenize (sentence)
  list_index=[]
  for i in range(len(tex)):
    list_index.append(sentence.index(tex[i]))
    #print(tex[i])
    #print(sentence.index(tex[i]))
  list_index.sort()
  txt=''
  for i in range(list_index[0],list_index[0]+len(list_index)):
    txt=txt + ' ' + sentence[i]
    #print(sentence[i])
  return txt



# **پردازش پرسش**

In [ ]:
#@title پرسش با پایگاه داده سئوالات
#@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@222
import csv
file = open('/content/drive/MyDrive/Thesis/out21-1.csv')

type(file)
csvreader = csv.reader(file)
header = []
header = next(csvreader)
header
rows = []


for row in csvreader:
        rows.append(row)
rows

def Comp(row,type_Q,list_Question,nsubj_com,obj_com,oblTime_com,oblPlace_com,oblManner_com,oblNull_com,verb_com):
  Result_sum=0
  Result_Ans=''
  N=0
  if list_Question[2]=='':
    nsubj_com=0
  else: N=N+1 
  if list_Question[3]=='':
    obj_com=0 
  else: N=N+1 
  if list_Question[4]=='':
    oblTime_com=0
  else: N=N+1  
  if list_Question[5]=='':
    oblPlace_com=0
  else: N=N+1  
  if list_Question[6]=='':
    oblManner_com=0
  else: N=N+1  
  if list_Question[7]=='':
    oblNull_com=0
  else: N=N+1  
  if list_Question[8]=='':
    verb_com=0
  else: N=N+1  

  N=N-1
  if type_Q == 'nsubj':
      #print('سئوال در خصوص فاعل می باشد')
      if row[2]:
        Result_sum = obj_com + oblTime_com + oblPlace_com + oblManner_com + oblNull_com + verb_com 
        Result_sum= Result_sum/N
        Result_Ans=row[2]
      else:
        Result_sum=0
        
  elif type_Q == "obj":
      #print('سئوال در خصوص مفعول می باشد')
      if row[3]:
        Result_sum = nsubj_com + oblTime_com + oblPlace_com + oblManner_com + oblNull_com + verb_com 
        Result_sum= Result_sum/N
        Result_Ans=row[3]
      else:
        Result_sum=0 

  elif type_Q == "oblTime":
      #print('سئوال در خصوص قید زمان می باشد' )
      if row[4]:
        Result_sum = nsubj_com + obj_com + oblPlace_com + oblManner_com + oblNull_com + verb_com 
        Result_sum= Result_sum/N
        Result_Ans=row[4]
      else:
        Result_sum=0

  elif type_Q == "oblPlace":
      #print('سئوال در خصوص قید مکان می باشد')
      if row[5]:
        Result_sum = nsubj_com + obj_com + oblTime_com  + oblManner_com + oblNull_com + verb_com 
        #print(nsubj_com , obj_com , oblTime_com  , oblManner_com , oblNull_com , verb_com )
        #print('olace:::::::::::::::::',Result_sum )
        Result_sum= Result_sum/N
        Result_Ans=row[5]

      else:
        Result_sum=0   

  elif type_Q == "oblManner":
      #print('سئوال در خصوص قید ... می باشد')
      if row[6]:
        Result_sum = nsubj_com + obj_com + oblTime_com + oblPlace_com + oblNull_com + verb_com 
        Result_sum= Result_sum/N
        Result_Ans=row[6]
      else:
        Result_sum=0

  elif type_Q == "verb":
      #print('سئوال در خصوص فعل می باشد')
      if row[8]:
        Result_sum = nsubj_com + obj_com + oblTime_com + oblPlace_com + oblManner_com + oblNull_com  
        Result_sum= Result_sum/N
        Result_Ans=row[8]
      else:
        Result_sum=0

  else:
      #print("Please choose correct answer")
      if row[7]:
        Result_sum = nsubj_com + obj_com + oblTime_com + oblPlace_com + oblManner_com + verb_com
        Result_sum= Result_sum/N
        Result_Ans=row[7] 
      else:
        Result_sum=0
  return Result_sum,Result_Ans
#@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@2

with open('/content/drive/MyDrive/Thesis/output1.csv', 'w', newline='') as file1, open('/content/drive/MyDrive/Thesis/output2.csv', 'w', newline='') as file2, open('/content/drive/MyDrive/Thesis/output3.csv', 'w', newline='') as file3:
    writer1 = csv.writer(file1)
    writer2 = csv.writer(file2)
    writer3 = csv.writer(file3)
    writer1.writerow(["SN", "Question", "Question_type"])
    writer2.writerow(["SN", "Question", "Answer"])
    writer3.writerow(["SN", "Question", "Candidate"])
    for row in range(len(rows)) :
      text=rows[row][0]
      text= text.replace("\u200c", " " )
      normalizer = Normalizer()
      text = normalizer.normalize(text)
      text1=sent_tokenize(text)
      #for sent in text1:
      #print("text1,text1",text)
      senn=[]
      flag='0'
      for sent in text1:
        #print('جمله:::::::',sent)
 
        id2, doc2, nsubj2, obj2, oblTime2, oblPlace2, oblManner2, oblNull12, verb2=quintuplethazm(sent,id,flag)
        list_Question=(id2, doc2, nsubj2, obj2, oblTime2, oblPlace2, oblManner2, oblNull12, verb2)
        #id, doc, nsubj, obj, oblTime, oblPlace, oblManner, oblNull1, verb=quintuplet(sent,id,flag)
        #print(id2, doc2, nsubj2, obj2, oblTime2, oblPlace2, oblManner2, oblNull12, verb2)

      #print(doc2, nsubj2, obj2, oblTime2, oblPlace2, oblManner2, oblNull12, verb2)
      question_list=['چه‌ جور','چیست','مکانی','چه سان','چگونه','چطور','چه\u200cطور','چه\u200cسان','چه اندازه','چه\u200cاندازه','چند','چه مقدار','چه\u200cقدر','چه\u200cمقدار','چندمی','چندم','چندمین','کجا','کدامی','چقدر','کدام','چه زمانی','چه وقت','چه','چه\u200cکسی' , 'چه\u200cفردی', 'چه\u200cشخصی', 'چه کسی' ,'چه\u200cکسانی' , 'چه\u200cافرادی', 'چه\u200cاشخاصی', 'چه کسانی','چه\u200cچیزی' , 'چه\u200cوسیله\u200cای', 'چه\u200cشی\u200cای', 'چه چیزی' , 'چه وسیله ای', 'چه شی ای']
      type_Q=''
      Question_type=''
     
      if bool([ele for ele in question_list if(ele in nsubj2)]):
        Question_type='سئوال در خصوص فاعل می باشد' 
        type_Q='nsubj'
   
      elif bool([ele for ele in question_list if(ele in obj2)]):
        Question_type='سئوال در خصوص مفعول می باشد' 
        type_Q='obj'
   
      elif bool([ele for ele in question_list if(ele in oblTime2)]):
        Question_type='سئوال در خصوص قید زمان می باشد' 
        type_Q='oblTime'
      elif bool([ele for ele in question_list if(ele in oblPlace2)]):
        Question_type='سئوال در خصوص قید مکان می باشد' 
        type_Q='oblPlace'
      elif bool([ele for ele in question_list if(ele in oblManner2)]):
        Question_type='سئوال در خصوص قید حالت می باشد' 
        type_Q='oblManner'
      elif bool([ele for ele in question_list if(ele in oblNull12)]):
        Question_type='سئوال در خصوص قید ... می باشد' 
        type_Q='oblNull1'
     
      elif bool([ele for ele in question_list if(ele in verb2)]):
        Question_type='سئوال در خصوص فعل می باشد' 
        type_Q='verb'

      #print(Question_type)
      #print(type_Q)


      writer1.writerow([row, rows[row][0], Question_type])


      conn = sqlite3.connect('/content/drive/MyDrive/Thesis/QandA.db')

      sen_embedding = model1.encode(text)
      #sen_embedding.shape

      list_id=[]
      

      # code adapted from https://github.com/UKPLab/sentence-transformers/blob/master/examples/application_semantic_search.py

      #text = 'سئوال را بپورسید ؟' #@param {type: 'string'}

      queries = [text]
      query_embeddings = model1.encode(queries)

      # Find the closest 3 sentences of the corpus for each query sentence based on cosine similarity
      number_top_matches = 5 

      #print("Semantic Search Results")

      for query, query_embedding in zip(queries, query_embeddings):
          distances = scipy.spatial.distance.cdist([query_embedding], sentence_embeddings, "cosine")[0]

          results = zip(range(len(distances)), distances)
          results = sorted(results, key=lambda x: x[1])

          #print("\n\n======================\n\n")
          #print("Query:", query)
          #print("\nTop 5 most similar sentences in corpus:")

          for idx, distance in results[0:number_top_matches]:
            # #print(idx,sentences[idx].strip(), "(Cosine Score: %.5f)" % (1-distance))
              list_id.append(idx+1)


      #print('list_id:::::::::::::::',list_id)
      Result_list=[]
      Result_Answer=[]
      Result_Sent=[]
      Result_sum=0
      for i in list_id:
        #print('iiiiiiiiiiiiiiii:::',i)
        cursor = conn.execute("SELECT id, sentence, nsubj, obj, oblTime, oblPlace, oblManner, oblNull, verb FROM knowledge_data_full where id=?",(i,))
        

        for row1 in cursor:
          #print('################',row)
          #print('id',row[0])
          #print('sentence',row[1])
          #print('nsubj',row[2])
          #print('obj',row[3])
          #print('oblTime',row[4])
          #print('oblPlace',row[5])
          #print('oblManner',row[6])
          #print('oblNull',row[7])
          #print('verb',row[8])

          if list_Question[2]!='' and row1[2]:
            nsubj_com=embedding_Comparison(list_Question[2],row1[2])
          else: nsubj_com=0 
          if list_Question[3]!='' and row1[3]:
            obj_com=embedding_Comparison(list_Question[3],row1[3])
          else: obj_com=0 
          if list_Question[4]!='' and row1[4]:
            oblTime_com=embedding_Comparison(list_Question[4],row1[4])
          else: oblTime_com=0 
          if list_Question[5]!='' and row1[5]:
            oblPlace_com=embedding_Comparison(list_Question[5],row1[5])
          else: oblPlace_com=0 
          if list_Question[6]!='' and row1[6]:
            #print('$$$$$$$$$$$$$$$$$$$$$$$$$$$$4444444')
            oblManner_com=embedding_Comparison(list_Question[6],row1[6])
          else: oblManner_com=0 
          if list_Question[7]!='' and row1[7]:
            oblNull_com=embedding_Comparison(list_Question[7],row1[7])
          else: oblNull_com=0 
          if list_Question[8]!='' and row1[8]:
            verb_com=embedding_Comparison(list_Question[8],row1[8])
          else: verb_com=0 

          Result_sum,Result_Ans=Comp(row1,type_Q,list_Question,nsubj_com,obj_com,oblTime_com,oblPlace_com,oblManner_com,oblNull_com,verb_com)
          #print("Result_sum : ", Result_sum)
          #print("Result_Ans : ", Result_Ans)
          Result_Sent
        Result_list.append(Result_sum)
        Result_Answer.append(Result_Ans)
        Result_Sent.append(row1[1])

      conn.close()

      #print(Question_type)
      #print(type_Q)
      #print(Result_sum)

      Result_list1=[]
      for res in Result_list:
        Result_list1.append(res)
      Result_list1.sort()
      #print("Largest element 1 is:", Result_list1[-1],">> Result Answer 1:",Result_Answer[Result_list.index(Result_list1[-1])]," >> sentence Answer 1:",Result_Sent[Result_list.index(Result_list1[-1])])
      #print("Largest element 2 is:", Result_list1[-2],">> Result Answer 2:",Result_Answer[Result_list.index(Result_list1[-2])]," >> sentence Answer 2:",Result_Sent[Result_list.index(Result_list1[-2])])
      #print("Largest element 3 is:", Result_list1[-3],">> Result Answer 3:",Result_Answer[Result_list.index(Result_list1[-3])]," >> sentence Answer 3:",Result_Sent[Result_list.index(Result_list1[-3])])
      #print("Largest element 3 is:", Result_list1[-4],">> Result Answer 3:",Result_Answer[Result_list.index(Result_list1[-4])]," >> sentence Answer 3:",Result_Sent[Result_list.index(Result_list1[-4])])
      #print("Largest element 3 is:", Result_list1[-5],">> Result Answer 3:",Result_Answer[Result_list.index(Result_list1[-5])]," >> sentence Answer 3:",Result_Sent[Result_list.index(Result_list1[-5])])

      #print("Largest element is:", max(Result_list))

      #print(rows[row][0])
      
    
      #print("Largest element 1 is:", Result_list1[-1],">> Result Answer 1:",wordcloud_def(Result_Answer[Result_list.index(Result_list1[-1])],stop_words)
      #," >> sentence Answer 1:",Result_Sent[Result_list.index(Result_list1[-1])])
      #print("Largest element 2 is:", Result_list1[-2],">> Result Answer 2:",wordcloud_def(Result_Answer[Result_list.index(Result_list1[-2])],stop_words)
      #," >> sentence Answer 2:",Result_Sent[Result_list.index(Result_list1[-2])])
      #print("Largest element 3 is:", Result_list1[-3],">> Result Answer 3:",wordcloud_def(Result_Answer[Result_list.index(Result_list1[-3])],stop_words)
      #," >> sentence Answer 3:",Result_Sent[Result_list.index(Result_list1[-3])])
      
      #print(row, rows[row][0])
      if Result_list1[-1]!=0:
        writer2.writerow([row, rows[row][0], Result_Answer[Result_list.index(Result_list1[-1])]])
      else:
        writer2.writerow([row, rows[row][0], "Not_Result"])
      if Result_list1[-1]!=0:
        writer3.writerow([row, rows[row][0], Result_Sent[Result_list.index(Result_list1[-1])]])
      if Result_list1[-2]!=0:
        writer3.writerow([row, rows[row][0], Result_Sent[Result_list.index(Result_list1[-2])]])
      if Result_list1[-3]!=0:
        writer3.writerow([row, rows[row][0], Result_Sent[Result_list.index(Result_list1[-3])]])
      if Result_list1[-4]!=0:
        writer3.writerow([row, rows[row][0], Result_Sent[Result_list.index(Result_list1[-4])]])
      if Result_list1[-5]!=0:
        writer3.writerow([row, rows[row][0], Result_Sent[Result_list.index(Result_list1[-5])]])



In [ ]:
#@title پاسخ به پرسش کاربر

#@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@222

def Comp(row,type_Q,list_Question,nsubj_com,obj_com,oblTime_com,oblPlace_com,oblManner_com,oblNull_com,verb_com):
  Result_sum=0
  Result_Ans=''
  N=0
  if list_Question[2]=='':
    nsubj_com=0
  else: N=N+1 
  if list_Question[3]=='':
    obj_com=0 
  else: N=N+1 
  if list_Question[4]=='':
    oblTime_com=0
  else: N=N+1  
  if list_Question[5]=='':
    oblPlace_com=0
  else: N=N+1  
  if list_Question[6]=='':
    oblManner_com=0
  else: N=N+1  
  if list_Question[7]=='':
    oblNull_com=0
  else: N=N+1  
  if list_Question[8]=='':
    verb_com=0
  else: N=N+1  

  N=N-1
  if type_Q == 'nsubj':
      #print('سئوال در خصوص فاعل می باشد')
      if row[2]:
        Result_sum = obj_com + oblTime_com + oblPlace_com + oblManner_com + oblNull_com + verb_com 
        Result_sum= Result_sum/N
        Result_Ans=row[2]
      else:
        Result_sum=0
        
  elif type_Q == "obj":
      #print('سئوال در خصوص مفعول می باشد')
      if row[3]:
        Result_sum = nsubj_com + oblTime_com + oblPlace_com + oblManner_com + oblNull_com + verb_com 
        Result_sum= Result_sum/N
        Result_Ans=row[3]
      else:
        Result_sum=0 

  elif type_Q == "oblTime":
      #print('سئوال در خصوص قید زمان می باشد' )
      if row[4]:
        Result_sum = nsubj_com + obj_com + oblPlace_com + oblManner_com + oblNull_com + verb_com 
        Result_sum= Result_sum/N
        Result_Ans=row[4]
      else:
        Result_sum=0

  elif type_Q == "oblPlace":
      #print('سئوال در خصوص قید مکان می باشد')
      if row[5]:
        Result_sum = nsubj_com + obj_com + oblTime_com  + oblManner_com + oblNull_com + verb_com 
        #print(nsubj_com , obj_com , oblTime_com  , oblManner_com , oblNull_com , verb_com )
        #print('olace:::::::::::::::::',Result_sum )
        Result_sum= Result_sum/N
        Result_Ans=row[5]

      else:
        Result_sum=0   

  elif type_Q == "oblManner":
      #print('سئوال در خصوص قید ... می باشد')
      if row[6]:
        Result_sum = nsubj_com + obj_com + oblTime_com + oblPlace_com + oblNull_com + verb_com 
        Result_sum= Result_sum/N
        Result_Ans=row[6]
      else:
        Result_sum=0

  elif type_Q == "verb":
      #print('سئوال در خصوص فعل می باشد')
      if row[8]:
        Result_sum = nsubj_com + obj_com + oblTime_com + oblPlace_com + oblManner_com + oblNull_com  
        Result_sum= Result_sum/N
        Result_Ans=row[8]
      else:
        Result_sum=0

  else:
      #print("Please choose correct answer")
      if row[7]:
        Result_sum = nsubj_com + obj_com + oblTime_com + oblPlace_com + oblManner_com + verb_com
        Result_sum= Result_sum/N
        Result_Ans=row[7] 
      else:
        Result_sum=0
  return Result_sum,Result_Ans
#@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@2


text1=[]
from hazm import sent_tokenize, word_tokenize
Text_Question = "\u062E\u062F\u0627\u0648\u0646\u062F \u0686\u0647 \u06A9\u0633\u0627\u0646\u06CC \u0631\u0627 \u062F\u0631 \u062F\u0631\u0648\u0646 \u0632\u0645\u06CC\u0646 \u0642\u0631\u0627\u0631 \u0645\u06CC\u062F\u0647\u062F" #@param {type: 'string'}
Text_Question= Text_Question.replace("\u200c", " " )

normalizer = Normalizer()
Text_Question = normalizer.normalize(Text_Question)
text1=sent_tokenize(Text_Question)
#for sent in text1:
print("text1,text1",Text_Question)
senn=[]
flag='0'
for sent in text1:
  print('جمله:::::::',sent)
  
  id2, doc2, nsubj2, obj2, oblTime2, oblPlace2, oblManner2, oblNull12, verb2=quintuplethazm(sent,id,flag)
  list_Question=(id2, doc2, nsubj2, obj2, oblTime2, oblPlace2, oblManner2, oblNull12, verb2)
  #id, doc, nsubj, obj, oblTime, oblPlace, oblManner, oblNull1, verb=quintuplet(sent,id,flag)
  #print(id2, doc2, nsubj2, obj2, oblTime2, oblPlace2, oblManner2, oblNull12, verb2)



print("doc2:",doc2, "nsubj2",nsubj2, "obj2",obj2, "oblTime2",oblTime2, "oblPlace2",oblPlace2, "oblManner2",oblManner2, "oblNull12",oblNull12, "verb2,",verb2,"####")
question_list=['چه‌ جور','چیست','مکانی','چه سان','چگونه','چطور','چه\u200cطور','چه\u200cسان','چه اندازه','چه\u200cاندازه','چند','چه مقدار','چه\u200cقدر','چه\u200cمقدار','چندمی','چندم','چندمین','کجا','کدامی','چقدر','کدام','چه زمانی','چه وقت','چه','چه\u200cکسی' , 'چه\u200cفردی', 'چه\u200cشخصی', 'چه کسی' ,'چه\u200cکسانی' , 'چه\u200cافرادی', 'چه\u200cاشخاصی', 'چه کسانی','چه\u200cچیزی' , 'چه\u200cوسیله\u200cای', 'چه\u200cشی\u200cای', 'چه چیزی' , 'چه وسیله ای', 'چه شی ای']
type_Q=''
Question_type=''
res = [ele for ele in question_list if(ele in nsubj2)]
if bool(res):
  Question_type='سئوال در خصوص فاعل می باشد' 
  type_Q='nsubj'
elif bool([ele for ele in question_list if(ele in obj2)]):
  Question_type='سئوال در خصوص مفعول می باشد' 
  type_Q='obj'
elif bool([ele for ele in question_list if(ele in oblTime2)]):
  Question_type='سئوال در خصوص قید زمان می باشد' 
  type_Q='oblTime'

elif bool([ele for ele in question_list if(ele in oblPlace2)]):
  Question_type='سئوال در خصوص قید مکان می باشد' 
  type_Q='oblPlace'

elif bool([ele for ele in question_list if(ele in oblManner2)]):
  Question_type='سئوال در خصوص قید حالت می باشد' 
  type_Q='oblManner'

elif bool([ele for ele in question_list if(ele in oblNull12)]):
  Question_type='سئوال در خصوص قید ... می باشد' 
  print(res)
  type_Q='oblNull1'

elif bool([ele for ele in question_list if(ele in verb2)]):
  Question_type='سئوال در خصوص فعل می باشد' 
  type_Q='verb'



print(Question_type)
print(type_Q)


conn = sqlite3.connect('/content/drive/MyDrive/Thesis/QandA.db')

sen_embedding = model1.encode(Text_Question)
#sen_embedding.shape

list_id=[]

# code adapted from https://github.com/UKPLab/sentence-transformers/blob/master/examples/application_semantic_search.py

#Text_Question = 'سئوال را بپورسید ؟' #@param {type: 'string'}

queries = [Text_Question]
query_embeddings = model1.encode(queries)

# Find the closest 3 sentences of the corpus for each query sentence based on cosine similarity
number_top_matches = 5 #@param {type: "number"}

print("Semantic Search Results")

for query, query_embedding in zip(queries, query_embeddings):
    distances = scipy.spatial.distance.cdist([query_embedding], sentence_embeddings, "cosine")[0]

    results = zip(range(len(distances)), distances)
    results = sorted(results, key=lambda x: x[1])

    print("\n\n======================\n\n")
    print("Query:", query)
    print("\nTop 5 most similar sentences in corpus:")

    for idx, distance in results[0:number_top_matches]:
        #print(idx,sentences[idx].strip(), "(Cosine Score: %.5f)" % (1-distance))
        list_id.append(idx+1)
        print(idx)


print('list_id:::::::::::::::',list_id)
Result_list=[]
Result_Answer=[]
Result_Sent=[]
Result_sum=0
for i in list_id:
  print('iiiiiiiiiiiiiiii:::',i)
  cursor = conn.execute("SELECT id, sentence, nsubj, obj, oblTime, oblPlace, oblManner, oblNull, verb FROM knowledge_data_full where id=?",(i,))
  

  for row in cursor:
    #print('################',row)
    #print('id',row[0])
    print('sentence',row[1])
    #print('nsubj',row[2])
    #print('obj',row[3])
    #print('oblTime',row[4])
    #print('oblPlace',row[5])
    #print('oblManner',row[6])
    #print('oblNull',row[7])
    #print('verb',row[8])

    if list_Question[2]!='' and row[2]:
      print(list_Question[2], row[2])
      nsubj_com=embedding_Comparison(list_Question[2],row[2])
    else: nsubj_com=0 
    if list_Question[3]!='' and row[3]:
      print(list_Question[3], row[3])
      obj_com=embedding_Comparison(list_Question[3],row[3])
    else: obj_com=0 
    if list_Question[4]!='' and row[4]:
      print(list_Question[4], row[4])
      oblTime_com=embedding_Comparison(list_Question[4],row[4])
    else: oblTime_com=0 
    if list_Question[5]!='' and row[5]:
      print(list_Question[5], row[5])
      oblPlace_com=embedding_Comparison(list_Question[5],row[5])
    else: oblPlace_com=0 
    if list_Question[6]!='' and row[6]:
      print(list_Question[6], row[6])
      #print('$$$$$$$$$$$$$$$$$$$$$$$$$$$$4444444')
      oblManner_com=embedding_Comparison(list_Question[6],row[6])
    else: oblManner_com=0 
    if list_Question[7]!='' and row[7]:
      print(list_Question[7], row[7])
      oblNull_com=embedding_Comparison(list_Question[7],row[7])
    else: oblNull_com=0 
    if list_Question[8]!='' and row[8]:
      print(list_Question[8], row[8])
      verb_com=embedding_Comparison(list_Question[8],row[8])
    else: verb_com=0 

    Result_sum,Result_Ans=Comp(row,type_Q,list_Question,nsubj_com,obj_com,oblTime_com,oblPlace_com,oblManner_com,oblNull_com,verb_com)
    print("Result_sum : ", Result_sum)
    print("Result_Ans : ", Result_Ans)
    Result_Sent
  Result_list.append(Result_sum)
  Result_Answer.append(Result_Ans)
  Result_Sent.append(row[1])

conn.close()

print(Question_type)
print(type_Q)
print(Result_sum)

Result_list1=[]
for res in Result_list:
  Result_list1.append(res)
Result_list1.sort()
#print("Largest element 1 is:", Result_list1[-1],">> Result Answer 1:",Result_Answer[Result_list.index(Result_list1[-1])]," >> sentence Answer 1:",Result_Sent[Result_list.index(Result_list1[-1])])
#print("Largest element 2 is:", Result_list1[-2],">> Result Answer 2:",Result_Answer[Result_list.index(Result_list1[-2])]," >> sentence Answer 2:",Result_Sent[Result_list.index(Result_list1[-2])])
#print("Largest element 3 is:", Result_list1[-3],">> Result Answer 3:",Result_Answer[Result_list.index(Result_list1[-3])]," >> sentence Answer 3:",Result_Sent[Result_list.index(Result_list1[-3])])
#print("Largest element is:", max(Result_list))

print("پاسخ سئوال شما : " , sort1(Result_Answer[Result_list.index(Result_list1[-1])],Result_Sent[Result_list.index(Result_list1[-1])]))


print("Largest element 1 is:", Result_list1[-1],">> Result Answer 1:",Result_Answer[Result_list.index(Result_list1[-1])]
," >> sentence Answer 1:",Result_Sent[Result_list.index(Result_list1[-1])])
print("Largest element 2 is:", Result_list1[-2],">> Result Answer 2:",Result_Answer[Result_list.index(Result_list1[-2])]
," >> sentence Answer 2:",Result_Sent[Result_list.index(Result_list1[-2])])
print("Largest element 3 is:", Result_list1[-3],">> Result Answer 3:",Result_Answer[Result_list.index(Result_list1[-3])]
," >> sentence Answer 3:",Result_Sent[Result_list.index(Result_list1[-3])])
print("Largest element 4 is:", Result_list1[-4],">> Result Answer 1:",Result_Answer[Result_list.index(Result_list1[-4])]
," >> sentence Answer 4:",Result_Sent[Result_list.index(Result_list1[-4])])
print("Largest element 5 is:", Result_list1[-5],">> Result Answer 2:",Result_Answer[Result_list.index(Result_list1[-5])]
," >> sentence Answer 5:",Result_Sent[Result_list.index(Result_list1[-5])])
print("Largest element 6 is:", Result_list1[-6],">> Result Answer 3:",Result_Answer[Result_list.index(Result_list1[-6])]
," >> sentence Answer 6:",Result_Sent[Result_list.index(Result_list1[-6])])
print("***************@@@@@@@@****************")


In [ ]:
Result_list1=[]
#print(Result_list)
for res in Result_list:
  Result_list1.append(res)
Result_list1.sort()
#print("Largest element 1 is:", Result_list1[-1],">> Result Answer 1:",Result_Answer[Result_list.index(Result_list1[-1])]," >> sentence Answer 1:",Result_Sent[Result_list.index(Result_list1[-1])])
#print("Largest element 2 is:", Result_list1[-2],">> Result Answer 2:",Result_Answer[Result_list.index(Result_list1[-2])]," >> sentence Answer 2:",Result_Sent[Result_list.index(Result_list1[-2])])
#print("Largest element 3 is:", Result_list1[-3],">> Result Answer 3:",Result_Answer[Result_list.index(Result_list1[-3])]," >> sentence Answer 3:",Result_Sent[Result_list.index(Result_list1[-3])])
#print("Largest element is:", max(Result_list))



#print("Largest element 1 is:", Result_list1[-1],">> Result Answer 1:",wordcloud_def(Result_Answer[Result_list.index(Result_list1[-1])],stop_words)
," >> sentence Answer 1:",Result_Sent[Result_list.index(Result_list1[-1])])
#print("Largest element 2 is:", Result_list1[-2],">> Result Answer 2:",wordcloud_def(Result_Answer[Result_list.index(Result_list1[-2])],stop_words)
," >> sentence Answer 2:",Result_Sent[Result_list.index(Result_list1[-2])])
#print("Largest element 3 is:", Result_list1[-3],">> Result Answer 3:",wordcloud_def(Result_Answer[Result_list.index(Result_list1[-3])],stop_words)
," >> sentence Answer 3:",Result_Sent[Result_list.index(Result_list1[-3])])



In [ ]:
#print(Result_list)
#print("Largest element is:", max(Result_list))
#print(Result_list[7])
#print(Result_Answer)
#print(Result_Sent[7])
##print(sort1(Result_Answer[8],Result_Sent[8]))